# 02. Semi-supervised Dataset Preparation

- Mục tiêu: tạo bộ dữ liệu giữ cả phần **chưa có nhãn AQI** (aqi_class = NaN) để dùng cho self-training/co-training.
- Đồng thời **giả lập thiếu nhãn trong TRAIN** (time-aware) để mini project có thể thử nhiều mức thiếu nhãn.

In [1]:
CLEANED_PATH = "data/processed/01_cleaned.parquet"
OUTPUT_SEMI_DATASET_PATH = "data/processed/02_dataset_for_semi.parquet"
CUTOFF = "2017-01-01"
# Tỷ lệ nhãn bị che (mask) trong tập TRAIN
# 0.95 nghĩa là 95% dữ liệu train sẽ bị ẩn nhãn, chỉ 5% có nhãn
# Dùng để giả lập kịch bản thiếu nhãn cho semi-supervised learning
LABEL_MISSING_FRACTION = 0.95
RANDOM_STATE = 42

In [2]:
from pathlib import Path
import sys
import pandas as pd

# Xác định PROJECT_ROOT trước khi import từ src
PROJECT_ROOT = Path(".").resolve()
if not (PROJECT_ROOT / "data").exists() and (PROJECT_ROOT.parent / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()
 
# Thêm PROJECT_ROOT vào sys.path để Python tìm được module src
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.semi_supervised_library import SemiDataConfig, mask_labels_time_aware

df = pd.read_parquet((PROJECT_ROOT / CLEANED_PATH).resolve())

cfg = SemiDataConfig(cutoff=CUTOFF, random_state=int(RANDOM_STATE))
df2 = mask_labels_time_aware(df, cfg, missing_fraction=float(LABEL_MISSING_FRACTION))

out_path = (PROJECT_ROOT / OUTPUT_SEMI_DATASET_PATH).resolve()
out_path.parent.mkdir(parents=True, exist_ok=True)
df2.to_parquet(out_path, index=False)

print("Saved:", out_path)
print("Rows:", len(df2))
print("Labeled ratio:", float(df2["is_labeled"].mean()) if "is_labeled" in df2.columns else None)

Saved: E:\dnu.khmt.1701.1771040029@gmail.com\AirGuard\data\processed\02_dataset_for_semi.parquet
Rows: 420768
Labeled ratio: 0.08671049129211347


In [3]:
import pandas as pd

# Hiển thị một vài dòng đầu tiên của dataset
df_semi = pd.read_parquet((PROJECT_ROOT / OUTPUT_SEMI_DATASET_PATH).resolve())
display(df_semi.head(50))

# Lưu file CSV theo quy tắc: 02_dataset_for_semi_sample.csv
df_semi.head(500).to_csv(PROJECT_ROOT / "data/processed/02_dataset_for_semi_sample.csv", index=False)

print(f"df_semi.shape(): {df_semi.shape}")
print("Saved sample to: data/processed/02_dataset_for_semi_sample.csv")

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,SO2_lag24,NO2_lag24,CO_lag24,O3_lag24,TEMP_lag24,PRES_lag24,DEWP_lag24,RAIN_lag24,WSPM_lag24,is_labeled
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5,6,2013,3,1,5,5.0,5.0,18.0,18.0,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
6,7,2013,3,1,6,3.0,3.0,18.0,32.0,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
7,8,2013,3,1,7,3.0,6.0,19.0,41.0,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8,9,2013,3,1,8,3.0,6.0,16.0,43.0,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
9,10,2013,3,1,9,3.0,8.0,12.0,28.0,400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


df_semi.shape(): (420768, 56)
Saved sample to: data/processed/02_dataset_for_semi_sample.csv
